Open Database and Login

In [15]:
from vaultdb import VaultDB
import getpass
database_name = "test"
app = VaultDB(database_name)
connection = app.clone(input("Enter your VaultDB username: "), getpass.getpass())

In [2]:
connection.execute("SELECT * FROM vaultdb_configs").fetchdf()

,database_name,database_oid,schema_name,schema_oid,config_name,config_oid,internal,temporary,config_value
0,test,1102,security,1107,application_name,1126,False,False,test
1,test,1102,security,1107,identity_pool_id,1132,False,False,us-east-1:31811334-7ce6-4b79-a3f9-626074f797ed
2,test,1102,security,1107,remote,1138,False,False,s3://test-data-440955376164/merged_data
3,test,1102,security,1107,remote_merge_path,1144,False,False,s3://test-public-storage-440955376164
4,test,1102,security,1107,user_pool_client_id,1150,False,False,6fhshjiil20m03iol4h7nqi1mo
5,test,1102,security,1107,user_pool_id,1156,False,False,us-east-1_p8ca1Bn5Z


In [3]:
connection.execute("select * from duckdb_secrets();").fetch_df()

,name,type,provider,persistent,storage,scope,secret_string
0,vaultdb,s3,config,False,memory,"[s3://, s3n://, s3a://]",name=vaultdb;type=s3;provider=config;serializa...


Downloaded Database is empty

In [4]:
connection.execute("SELECT * FROM another_T").fetchdf()

,col1,col2,col3,col4,col5,col6,col7,col8


Data Inheritance

In [5]:
connection.execute(f"PRAGMA enable_data_inheritance;")
connection.execute("SELECT * FROM another_T").fetchdf()

,col1,col2,col3,col4,col5,col6,col7,col8


Insert Data Locally

In [6]:
connection.execute("INSERT INTO another_T VALUES (81,92,93,94,95,96,97,98)")

In [7]:
connection.execute(f"PRAGMA disable_data_inheritance;")
connection.execute("SELECT * FROM another_T").fetchdf()

,col1,col2,col3,col4,col5,col6,col7,col8
0,81,92,93,94,95,96,97,98


Push Inserted Data to Server

In [9]:
connection.execute(f'PUSH DATABASE {database_name};').fetchdf()

,Success


Data Inheritence

In [10]:
connection.execute(f"PRAGMA enable_data_inheritance;")
connection.execute("SELECT * FROM another_T").fetchdf()

,col1,col2,col3,col4,col5,col6,col7,col8
0,81,92,93,94,95,96,97,98
1,81,92,93,94,95,96,97,98


Truncate local table

In [11]:
connection.execute(f"TRUNCATE DATABASE {database_name};")
connection.execute("SELECT * FROM another_T").fetchdf()

,col1,col2,col3,col4,col5,col6,col7,col8
0,81,92,93,94,95,96,97,98


Create Objects Locally

In [12]:
connection.execute('BEGIN TRANSACTION;')
connection.execute('CREATE TABLE demo2 (col1 INT, col2 INT, col3 INT, col4 INT, col5 INT, col6 INT, col7 INT, col8 INT)')
connection.execute('ALTER TABLE demo2 PARTITION BY col1;')
connection.execute("INSERT INTO demo2 VALUES (1,2,3,4,5,6,7,8), (1,12,13,14,15,16,17,18), (1,20,30,40,50,60,70,80), (11,22,33,44,55,66,77,88), (111,222,333,444,555,666,777,888), (1111,2222,3333,4444,5555,6666,7777,8888)")
connection.execute('COMMIT;')

Push schema changes

In [13]:
connection.execute(f"PRAGMA enable_data_inheritance;")
connection.execute("SELECT * FROM demo2").fetchdf()

,col1,col2,col3,col4,col5,col6,col7,col8
0,1,2,3,4,5,6,7,8
1,1,12,13,14,15,16,17,18
2,1,20,30,40,50,60,70,80
3,11,22,33,44,55,66,77,88
4,111,222,333,444,555,666,777,888
5,1111,2222,3333,4444,5555,6666,7777,8888


In [14]:
connection.execute(f"PUSH DATABASE {database_name};").fetchdf()

BinderException: Binder Error: Catalog "dev" does not exist!

In [ ]:
connection.execute(f"TRUNCATE DATABASE {database_name};")

In [ ]:
connection.execute(f"PRAGMA enable_data_inheritance;")
connection.execute("SELECT * FROM demo2").fetchdf()